In [ ]:
from fenics import *
import numpy as np
from braininversion.IOHandling import (read_mesh_from_h5, write_to_xdmf, 
                                       xdmf_to_unstructuredGrid, read_xdmf_timeseries)
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                                           plot_pressures_and_forces_cross_section,
                                           extract_cross_section, style_dict)
import matplotlib.pyplot as plt
import yaml


In [ ]:
mesh_name = "ideal_brain_3D_N50"
T = 0.2
num_steps = 5
sim_name = f"{mesh_name}_{T}_{num_steps}"
config_file_path = f"../results/{sim_name}/{sim_name}_config.yml"
sim_file = f"../results/{sim_name}/{sim_name}_checkp.xdmf"
mesh_file = f"../meshes/{mesh_name}/{mesh_name}.xdmf"
boundary_file = f"../meshes/{mesh_name}/{mesh_name}_boundaries.xdmf"


with open(config_file_path) as conf_file:
    config = yaml.load(conf_file, Loader=yaml.FullLoader)

mesh_dir = config["mesh_dir_name"]
mesh_config_path = f"../meshes/{mesh_dir}/{mesh_dir}_config.yml"
with open(mesh_config_path) as conf_file:
    mesh_config = yaml.load(conf_file, Loader=yaml.FullLoader)


In [ ]:
ventricle_probe = [Point(mesh_config["ventricle_probe"])]
sas_probe = [Point(mesh_config["sas_probe"])]
parenchyma_probe = [Point(mesh_config["parenchyma_probe"])]

mmHg2Pa = 132.32
spinal_outlet_id = 3


dt = T/num_steps
times = np.linspace(dt, T, num_steps)
infile_mesh = XDMFFile(mesh_file)
mesh = Mesh()
infile_mesh.read(mesh)
gdim = mesh.geometric_dimension()

boundary_marker = MeshFunction("size_t", mesh, gdim - 1, 0)
boundary_infile = XDMFFile(boundary_file)
boundary_infile.read(boundary_marker)
boundary_infile.close()

In [ ]:
V = VectorFunctionSpace(mesh, "CG", 1)
W = FunctionSpace(mesh, "CG", 1)
names = {"pF":W, "pP":W, "phi":W,
         "d":V, "u":V
        }
infile_results = XDMFFile(sim_file)
results = {n:[] for n in names}
for n, space in names.items():
    for i in range(num_steps):
        f = Function(space)
        infile_results.read_checkpoint(f, n, i)
        results[n].append(f)

In [ ]:
pP_series = results["pP"]
pF_series = results["pF"]
phi_series = results["phi"]

pF_sas = extract_cross_section(pF_series, sas_probe).flatten()/mmHg2Pa
pF_ventricle = extract_cross_section(pF_series, ventricle_probe).flatten()/mmHg2Pa
pP_parenchyma = extract_cross_section(pP_series, parenchyma_probe).flatten()/mmHg2Pa
phi_parenchyma = extract_cross_section(phi_series, parenchyma_probe).flatten()/mmHg2Pa


plt.figure(figsize=(10,8))
plt.plot(times, pF_ventricle, label="ventricle")
plt.plot(times, pP_parenchyma, label="parenchyma fluid")
plt.plot(times, phi_parenchyma, label="parenchyma tot")
plt.plot(times, pF_sas, label="SAS")

plt.legend()
plt.grid()
plt.xlabel("t [s]")
plt.ylabel("p in mmHg")

In [ ]:
# compute outflow into spinal coord 

ds_outflow = Measure("ds", domain=mesh, subdomain_data=boundary_marker, subdomain_id=spinal_outlet_id)
n = FacetNormal(mesh)

m3tomL = 1e6
outflow = np.array([assemble(dot(u,n)*ds_outflow) for u in results["u"]])
plt.figure(figsize=(10,8))
plt.plot(times, outflow/dt*m3tomL, label="outflow into spinal coord")
plt.legend()
plt.grid()
plt.xlabel("time in s")
plt.ylabel("flowrate in mL/ s")

In [ ]:
from pathlib import Path
import pyvista as pv

In [ ]:
sim_file = f"../results/{sim_name}/{sim_name}.xdmf"

grid = xdmf_to_unstructuredGrid(sim_file, idx=[0,1])


In [ ]:
slices = grid.slice_orthogonal()
p = pv.Plotter()
p.add_mesh(slices,cmap="coolwarm")
p.show()

In [ ]:
clipped_data = grid.clip(normal=[0,1,0])
p = pv.PlotterITK()
p.add_mesh(clipped_data)
p.show()

In [ ]:
interpolate?